### 08 Agentic RAG Mainルーチン（Clean Architecture）

**`src/` 配下の Clean Architecture モジュールを `DIContainer` 経由で使用する Main ルーチン。**

gpt-oss:20b を使用する構成のため、**Colab GPU は L4 を使用すること。**

#### アーキテクチャ
- **Domain 層**: ドメインモデル、Port（Protocol）、`WorkflowConfig`
- **Use Cases 層**: `AgentWorkflow`（LangGraph）、`DataIngestion`、各ノード
- **Interface Adapters 層**: `OllamaAdapter`、`ChromaDBAdapter`、`RerankerAdapter`、`PDFLoaderAdapter`、`GradioHandler`
- **Infrastructure 層**: `DIContainer`（依存性の組み立て・注入）

#### ワークフロー
1. **task_planning**: ユーザの質問を分析し、サブタスク（目的＋検索クエリ）を作成
2. **doc_search**: ハイブリッド検索（BM25 + ベクトル）+ Reranker
3. **summarize**: 検索結果を要約（Judge の入力コンテキスト削減）
4. **judge**: 情報の十分性を判定。不足なら doc_search に戻る
5. **generate_answer**: 最終回答をストリーミング生成

**必要なライブラリをインストール**

In [1]:
# Google Colab に必要なライブラリをインストールする。
# NOTE: Colab では uv ではなく pip を使う。
#       uv は依存解決の過程で numpy 等をアップグレードし、
#       プリインストール済みの scipy 等を壊すため。

# fmt: off
pkgs = [
    "ollama", "langchain-ollama",
    "langchain>=1.2.8", "langchain-core>=1.2.8", "langgraph>=1.0.7",
    "langchain-text-splitters",
    "markitdown[all]", "chromadb", "rank-bm25",
    "spacy", "ginza", "ja-ginza", "sentence-transformers",
    "pydantic>=2.12", "pydantic-settings>=2.13",
    "gradio>=6.0",
]
# fmt: on

%pip install -U -q {" ".join(pkgs)}

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-http 1.38.0 requires opentelemetry-exporter-otlp-proto-common==1.38.0, but you have opentelemetry-exporter-otlp-proto-common 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.38.0 requires opentelemetry-proto==1.38.0, but you have opentelemetry-proto 1.39.1 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.38.0 requires opentelemetry-sdk~=1.38.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.
opentelemetry-exporter-gcp-logging 1.11.0a0 requires opentelemetry-sdk<1.39.0,>=1.35.0, but you have opentelemetry-sdk 1.39.1 which is incompatible.


In [2]:
# GiNZA (ja_ginza) は pip install ginza ja-ginza で自動的にインストールされるため、
# spacy download は不要。以下で読み込みを確認する。
import spacy

nlp = spacy.load("ja_ginza")
print(f"ja_ginza loaded: {nlp.meta['name']}")
del nlp

ja_ginza loaded: ginza


**Google Colab に Ollama をセットアップ**
- 詳細は [01_connect_oss_llm.ipynb](01_connect_oss_llm.ipynb) を参照。

In [3]:
# Ollama のインストール・起動・モデルのダウンロード
import subprocess
import time
import ollama  # type: ignore

!apt-get install -y -qq zstd
!curl -fsSL https://ollama.com/install.sh | sh

process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)
time.sleep(5)


def ollama_pull(model: str) -> None:
    """Ollama モデルをダウンロードし、進捗をインライン表示する。"""
    for progress in ollama.pull(model, stream=True):
        status = progress.get("status", "")
        total = progress.get("total") or 0
        completed = progress.get("completed") or 0
        if total:
            line = f"{status}: {completed / total:.0%}"
        else:
            line = status
        print(f"\r{line:<60}", end="", flush=True)
    print(f"\n{model}: Done!")


model_name = "gpt-oss:20b"
ollama_pull(model_name)
!ollama show {model_name}

Selecting previously unselected package zstd.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
success                                                     
gpt-oss:20b: Done!
  Model
    architecture        gptoss    
    parameters          20.9B     
    context length      131072    
    embedding length    2880      
    quantization        MXFP4     

  Ca

**Google Drive から `src/` をローカルにコピーし、Python パスに追加**

Google Drive マウント経由の import はネットワークアクセスが毎回発生して遅いため、
起動時に `src/` を Colab ローカル（`/content/src/`）にコピーして、そこから import する。

In [4]:
import shutil
import sys
from pathlib import Path

from google.colab import drive  # type: ignore

# Google Drive をマウント
drive.mount("/content/drive")

# Google Drive の src/ を Colab ローカルにコピー（ネットワーク遅延の回避）
drive_src = Path("/content/drive/MyDrive/GEN_AI_RAG/src")
local_src = Path("/content/src")

if local_src.exists():
    shutil.rmtree(local_src)
shutil.copytree(drive_src, local_src)

# ローカルコピーを Python パスに追加
if str(local_src) not in sys.path:
    sys.path.insert(0, str(local_src))

print(f"src copied: {drive_src} -> {local_src}")
for pkg in ["domain", "usecases", "interfaces", "infrastructure"]:
    print(f"  {pkg}/: {(local_src / pkg / '__init__.py').exists()}")

Mounted at /content/drive
src copied: /content/drive/MyDrive/GEN_AI_RAG/src -> /content/src
  domain/: True
  usecases/: True
  interfaces/: True
  infrastructure/: True


**DIContainer で依存性を組み立て、Gradio UI を起動**

Clean Architecture の Infrastructure 層にある `DIContainer` が、
すべての依存性（LLM・VectorStore・Reranker・DataLoader）を生成し、
コンストラクタインジェクションで `AgentWorkflow` と `GradioHandler` に注入する。

`WorkflowConfig` のハイパーパラメータは、環境変数（`RAG_` プレフィックス）で上書き可能。

In [5]:
import logging

from domain.config import WorkflowConfig
from infrastructure.di_container import DIContainer

# ログ設定
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
)

# 設定を生成（デフォルト値または環境変数から読み込み）
config = WorkflowConfig()

# DI コンテナで全依存性を組み立て
container = DIContainer(config=config)

# Gradio UI を生成・起動
ui = container.create_ui()
demo = ui.launch()
demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f953e1c24e273f853.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
demo.close()

Closing server running on port: 7860
